In [1]:
%reload_ext autoreload
%autoreload 1
import sys
sys.path.insert(0, "../")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from utils.loader import load

### Load dataset and normalize

In [2]:
# load the dataset
train_input,train_target, train_classes, test_input, test_target, test_classes = load()

In [3]:
#Normalize the dataset
mean, std = train_input.mean(), train_input.std()

train_input = train_input.sub_(mean).div_(std)
test_input = test_input.sub_(mean).div_(std)

### Models

In [4]:
### Digit recognition and digit comparison in a parralel manner manner
class LeNet_aux_parralel(nn.Module):
    def __init__(self):
        super(LeNet_aux_parralel, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size = 3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 2)
        self.conv3 = nn.Conv2d(1,32,kernel_size=3)
        self.fc1 = nn.Linear(64, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 2)
        self.fc4 = nn.Linear(64,128)
        self.fc5 = nn.Linear(128,10)
         
    def forward(self, x):
        y = x.view(-1,1,14,14)
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 2))
        x = F.relu(self.fc1(x.view(-1,64)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        y = F.relu(F.max_pool2d(self.conv3(y), kernel_size = 3))
        y = F.relu(F.max_pool2d(self.conv2(y), kernel_size = 2))
        y = F.relu(self.fc4(y.view(-1,64)))
        y = F.relu(self.fc5(y))
        return x,y

In [61]:
### Digit recognition and digit comparison in a sequential manner manner
class LeNet_aux_sequential(nn.Module):
    def __init__(self):
        super(LeNet_aux_sequential, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        
        self.fc3 = nn.Linear(20, 60)
        self.fc4 = nn.Linear(60, 90)
        self.fc5 = nn.Linear(90, 2)
        
    def forward(self, input_):        
        
        x = input_[:, 0, :, :].view(-1, 1, 14, 14)
        y = input_[:, 1, :, :].view(-1, 1, 14, 14)
        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        
        y = F.relu(F.max_pool2d(self.conv1(y), kernel_size=2, stride=2))
        y = F.relu(F.max_pool2d(self.conv2(y), kernel_size=2, stride=2))
        y = F.relu(self.fc1(y.view(-1, 256)))
        y = self.fc2(y)

        z = torch.cat([x, y], 1)
        
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = self.fc5(z)
        
        return x, y, z

### Training

In [41]:
# trainig function for the parralel implementation
def train_aux_parralel(model,train_input,train_target,train_classes, criterion = nn.MSELoss(),
              optimizer =optim.SGD,batch_size=100,eta = 0.1, nb_epochs = 100) :
    
    # Define the loss and optimizing algorithm
    optimizer = optimizer(model.parameters(), lr = eta)
    # Train the model
    for e in range(nb_epochs):
        running_loss = 0.0
        for input_, target1,target2 in (zip(train_input.split(batch_size), train_target.split(batch_size),train_classes.split(batch_size))):
            outx,outy = model(input_)
            loss1 = criterion(outx, target1)
            loss2 = criterion(outy,target2.view(200,-1).squeeze())
            loss = loss1 + loss2
            running_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('Epoch: {}   Loss {:.6f}'.format(e,running_loss ))

In [43]:
def train_aux_sequential(model, train_input, train_target, train_class,criterion = nn.CrossEntropyLoss(),
                       optimizer =optim.SGD,batch_size=100,eta = 0.1, nb_epochs = 100) :
    
    # Define the loss and optimizing algorithm
    optimizer = optimizer(model.parameters(), lr = eta)
    # Train the model
    for e in range(nb_epochs):
        running_loss = 0.0
        for input_, target_class in (zip(train_input.split(batch_size),train_classes.split(batch_size))):
            outx, outy, _ = model(input_)
            loss1 = criterion(outx, target_class[:,0])
            loss2 = criterion(outy, target_class[:,1])
            loss = loss1 + loss2
            running_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('Epoch: {}   Loss {:.6f}'.format(e,running_loss ))
        
    # Training of the comparison net      
    for e in range(nb_epochs):
        running_loss = 0.0
        for input_, target in (zip(train_input.split(batch_size), train_target.split(batch_size))): 
            _, _, out_z = model(input_)
            loss = criterion(out_z, target)
            running_loss += loss
            optimizer.zero_grad()
            loss_z.backward()
            optimizer.step() 
        print('Epoch: {}   Loss {:.6f}'.format(e,running_loss ))

### Errors

In [50]:
def compute_nb_errors_aux_parralel(model, data_input, data_target,data_class, criterion = nn.MSELoss()) :
    nb_errors,batch_size = 0, 100
    test_loss = 0.0
    for input_, target,class_ in (zip(data_input.split(batch_size), data_target.split(batch_size), data_class.split(batch_size))):
        outx, outy = model(input_)
        loss1 = criterion(outx, target)
        loss2 = criterion(outy,class_.view(200,-1).squeeze())
        loss = loss1 + loss2
        test_loss+= loss.item()
        _, predicted_classes = outx.max(1)
        for k in range(batch_size):
            if target[k] != predicted_classes[k]:
                nb_errors = nb_errors + 1
                
    print('test error Net {:0.2f}% {:d}/{:d} , test loss {:0.2f}'.format((100 * nb_errors) / data_input.size(0),nb_errors, data_input.size(0),test_loss))

In [62]:
def compute_nb_errors_aux_sequential(model, data_input, data_target, criterion = nn.MSELoss()) :
    nb_errors,batch_size = 0, 100
    test_loss = 0.0
    for input_, target in (zip(data_input.split(batch_size), data_target.split(batch_size))):
        _, _, outz = model(input_)
        loss = criterion(outz, target)
        test_loss+= loss.item()
        _, predicted_classes = outz.max(1)
        for k in range(batch_size):
            if target[k] != predicted_classes[k]:
                nb_errors = nb_errors + 1
                
    print('test error Net {:0.2f}% {:d}/{:d} , test loss {:0.2f}'.format((100 * nb_errors) / data_input.size(0),nb_errors, data_input.size(0),test_loss))

### Main

In [23]:
### Parralel implementation computation
model_aux_parralel = LeNet_aux_parralel()
train_aux(model_aux_parralel,train_input,train_target,train_classes,nn.CrossEntropyLoss(),nb_epochs=50)

Epoch: 0   Loss 29.838446
Epoch: 1   Loss 29.539350
Epoch: 2   Loss 29.181394
Epoch: 3   Loss 28.666164
Epoch: 4   Loss 27.882211
Epoch: 5   Loss 27.119325
Epoch: 6   Loss 26.356133
Epoch: 7   Loss 25.208215
Epoch: 8   Loss 23.217631
Epoch: 9   Loss 22.317188
Epoch: 10   Loss 20.023574
Epoch: 11   Loss 20.391545
Epoch: 12   Loss 18.221893
Epoch: 13   Loss 17.013589
Epoch: 14   Loss 15.758462
Epoch: 15   Loss 16.449066
Epoch: 16   Loss 14.869087
Epoch: 17   Loss 14.532806
Epoch: 18   Loss 15.120954
Epoch: 19   Loss 14.421781
Epoch: 20   Loss 12.819234
Epoch: 21   Loss 12.475372
Epoch: 22   Loss 12.355291
Epoch: 23   Loss 13.272635
Epoch: 24   Loss 11.307860
Epoch: 25   Loss 10.355968
Epoch: 26   Loss 11.571531
Epoch: 27   Loss 10.949960
Epoch: 28   Loss 10.339297
Epoch: 29   Loss 10.122843
Epoch: 30   Loss 10.209159
Epoch: 31   Loss 10.904464
Epoch: 32   Loss 9.844938
Epoch: 33   Loss 9.522384
Epoch: 34   Loss 8.577805
Epoch: 35   Loss 9.251294
Epoch: 36   Loss 9.894008
Epoch: 37   Loss

In [51]:
nb_test_errors_aux_parralel = compute_nb_errors_aux_parralel(model_aux_parralel, test_input, test_target,test_classes,nn.CrossEntropyLoss())

test error Net 19.20% 192/1000 , test loss 9.37


In [47]:
### Sequential implementation computation
model_aux_sequential = LeNet_aux_sequential()
train_aux_sequential(model_aux_sequential,train_input,train_target,train_classes,nb_epochs=50)

torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 0   Loss 43.240405
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 1   Loss 35.589499
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 2   Loss 24.055720
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 3   Loss 19.131519
torch.Size([100,

torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 33   Loss 0.277782
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 34   Loss 0.262181
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 35   Loss 0.246513
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 36   Loss 0.233550
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100,

torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 16   Loss 0.140600
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 17   Loss 0.111294
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 18   Loss 0.088832
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
Epoch: 19   Loss 0.073002
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100,

torch.Size([100, 20])
Epoch: 49   Loss 0.006306


In [63]:
compute_nb_errors_aux_sequential(model_aux_sequential, test_input, test_target, criterion = nn.CrossEntropyLoss())

torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
torch.Size([100, 20])
test error Net 7.00% 70/1000 , test loss 3.32


In [64]:
### Sequential implementation computation
model_aux_sequential2 = LeNet_aux_sequential()
train_aux_sequential(model_aux_sequential2,train_input,train_target,train_classes,nb_epochs=50)
compute_nb_errors_aux_sequential(model_aux_sequential2, test_input, test_target, criterion = nn.CrossEntropyLoss())

Epoch: 0   Loss 43.195895
Epoch: 1   Loss 36.667513
Epoch: 2   Loss 35.473318
Epoch: 3   Loss 24.611356
Epoch: 4   Loss 15.895283
Epoch: 5   Loss 11.187095
Epoch: 6   Loss 7.613829
Epoch: 7   Loss 7.125373
Epoch: 8   Loss 4.428206
Epoch: 9   Loss 3.607456
Epoch: 10   Loss 3.213301
Epoch: 11   Loss 3.043544
Epoch: 12   Loss 3.250928
Epoch: 13   Loss 2.239110
Epoch: 14   Loss 1.861693
Epoch: 15   Loss 1.692763
Epoch: 16   Loss 1.528736
Epoch: 17   Loss 1.373706
Epoch: 18   Loss 1.235348
Epoch: 19   Loss 1.118620
Epoch: 20   Loss 1.014711
Epoch: 21   Loss 0.910129
Epoch: 22   Loss 0.813605
Epoch: 23   Loss 0.730242
Epoch: 24   Loss 0.650634
Epoch: 25   Loss 0.580164
Epoch: 26   Loss 0.520897
Epoch: 27   Loss 0.473008
Epoch: 28   Loss 0.428082
Epoch: 29   Loss 0.394162
Epoch: 30   Loss 0.364025
Epoch: 31   Loss 0.336318
Epoch: 32   Loss 0.312674
Epoch: 33   Loss 0.293348
Epoch: 34   Loss 0.274611
Epoch: 35   Loss 0.257947
Epoch: 36   Loss 0.242831
Epoch: 37   Loss 0.229284
Epoch: 38   Loss